In [ ]:
import os
import click
import pprint
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

input_folder = '/home/honey/cern/July/datasets/bsm/bsm_full/'
# input_folder = '/home/honey/cern/July/datasets/sm/single_higgs_10fb.csv'

file_list = os.listdir(input_folder)
file_list.sort()
# file_list = ['ww_10fb.csv']


for file in file_list:
    input_path = os.path.join(input_folder, file) 

    data = []    
    print('Reading data at ', input_path)
    with open(input_path, 'r') as file:
        for line in file.readlines():
            line = line.replace(';', ',')
            line = line.rstrip(',\n')
            line = line.split(',')
            data.append(line)

    #Find the longest line in the data 
    longest_line = max(data, key = len)
    
    #Set the maximum number of columns
    max_col_num = len(longest_line)

    #Set the columns names
    col_names = ['event_ID', 'process_ID', 'event_weight', 'MET', 'MET_Phi']
    meta_cols = col_names.copy()

    for i in range(1, (int((max_col_num-5)/5))+1):
        col_names.append('obj'+str(i))
        col_names.append('E'+str(i))
        col_names.append('pt'+str(i))
        col_names.append('eta'+str(i))
        col_names.append('phi'+str(i))

    #Create a dataframe from the list, using the column names from before

    print('Processing the data..')
    df = pd.DataFrame(data, columns=col_names)
    df.fillna(value=np.nan, inplace=True)

    x_train_df = pd.DataFrame(df.values, columns=col_names)
    x_train_df.fillna(value=0, inplace=True)

    x_train_df = x_train_df.drop(columns=meta_cols)
    
    from collections import OrderedDict 
    particle_list = {}
    for i in range(len(x_train_df)):
        for j in x_train_df.loc[i].keys():
            if 'obj' in j:
                if x_train_df.loc[i][j] not in particle_list.keys():
                    particle_list[x_train_df.loc[i][j]] = 1
                else:
                    particle_list[x_train_df.loc[i][j]] += 1
    particle_list.pop(0)
    print(particle_list)
    
    particle_list_sorted = OrderedDict(sorted(particle_list.items(), key = lambda x:x[1], reverse=True))
    
    print(dict(particle_list_sorted.items()))
    
    save = True
    # plt.bar(range(len(particle_list_sorted)), list(particle_list_sorted.values()), align='center')
    # plt.xticks(range(len(particle_list_sorted)), list(particle_list_sorted.keys()))

    x = list(particle_list_sorted.keys())
    y = list(particle_list_sorted.values())

    N = len(x)

    ind = np.arange(N)  # the x locations for the groups
    width = 0.75       # the width of the bars

    fig, ax = plt.subplots(figsize=(8,5), dpi = 100)
    rects1 = ax.bar(ind, y, width)

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Number of particles')
    plt.xticks(range(len(particle_list_sorted)), list(particle_list_sorted.keys()))
    plt.title(input_path.split('/')[-1], fontsize=15)
    
    total = np.sum(y)
    particle_percentage = {}
    for key in particle_list_sorted.keys():
        particle_percentage[key] = '%.2f'%((particle_list_sorted[key] * 100) / total)
    
    plt.xlabel(particle_percentage, fontsize=12)

    def autolabel(rects):
        """
        Attach a text label above each bar displaying its height
        """
        for rect in rects:
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., 1*height,
                    '%d' % int(height),
                    ha='center', va='bottom')

    autolabel(rects1)
    if save:
        plt.savefig(input_path.split('/')[-1][:-4] + '_particle_distribution.png')

In [ ]:
def plot_all(data, logy=True):
    for key in data.keys():
        plt.figure()
        plt.hist(data[key], bins=200)
        plt.xlabel(str(key))
        plt.ylabel('Number of particles')
        plt.yscale('log')

In [ ]:
import os
import click
import pprint
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

input_folder = '/home/honey/cern/July/datasets/sm/'
# input_folder = '/home/honey/cern/July/datasets/sm/single_higgs_10fb.csv'

# file_list = os.listdir(input_folder)
# file_list.sort()
file_list = ['zw_10fb.csv']
save = True

for file in file_list:
    input_path = os.path.join(input_folder, file) 

    data = []    
    print('Reading data at ', input_path)
    with open(input_path, 'r') as file:
        for line in file.readlines():
            line = line.replace(';', ',')
            line = line.rstrip(',\n')
            line = line.split(',')
            data.append(line)

    #Find the longest line in the data 
    longest_line = max(data, key = len)
    
    #Set the maximum number of columns
    max_col_num = len(longest_line)

    #Set the columns names
    col_names = ['event_ID', 'process_ID', 'event_weight', 'MET', 'MET_Phi']
    meta_cols = col_names.copy()

    for i in range(1, (int((max_col_num-5)/5))+1):
        col_names.append('obj'+str(i))
        col_names.append('E'+str(i))
        col_names.append('pt'+str(i))
        col_names.append('eta'+str(i))
        col_names.append('phi'+str(i))

    #Create a dataframe from the list, using the column names from before

    print('Processing the data..')
    df = pd.DataFrame(data, columns=col_names)
    df.fillna(value=np.nan, inplace=True)

    x_train_df = pd.DataFrame(df.values, columns=col_names)
    x_train_df.fillna(value=0, inplace=True)

    x_train_df = x_train_df.drop(columns=meta_cols)
    x = x_train_df.values.reshape([x_train_df.shape[0]*x_train_df.shape[1]//5,5])
    del x_train_df
    lst = []
    for i in range(x.shape[0]):
        if (x[i] == 0).all():
            lst.append(i)
    data = np.delete(x, lst, 0)
    col_names = ['obj', 'E', 'pt', 'eta', 'phi']
    data = pd.DataFrame(data, columns=col_names)
    data_4d_df = data.drop(columns='obj')
    data_4d_df = data_4d_df.fillna(0)
    data_4d_df = data_4d_df.astype('float32')
    data_4d_df['E'] = data_4d_df['E'] / 1000.0
    data_4d_df['pt'] = data_4d_df['pt'] / 1000.0
    
#     from collections import OrderedDict 
#     particle_list = {}
#     for i in range(len(x_train_df)):
#         for j in x_train_df.loc[i].keys():
#             if 'obj' in j:
#                 if x_train_df.loc[i][j] not in particle_list.keys():
#                     particle_list[x_train_df.loc[i][j]] = 1
#                 else:
#                     particle_list[x_train_df.loc[i][j]] += 1
    
#     plot_all(data_4d_df)
    for key in data_4d_df.keys():
        plt.figure()
        plt.hist(data_4d_df[key], bins=200)
        plt.xlabel(str(key))
        plt.ylabel('Number of particles')
        plt.yscale('log')
            
            
        if save:
            plt.savefig(input_path.split('/')[-1][:-4] + '_' + str(key) + '_data_distribution.png')